# Import libraries

In [1]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load and preprocess images

In [2]:
direction_classes = ["accelerate", "reverse", "leftTurn", "rightTurn", "leftInPlaceTurn", "rightInPlaceTurn", "stop"]

directory_path = r"C:\Users\Tomas Tomcany\PycharmProjects\rc_controller\captured_images"

image_paths = []
directions = []

# Iterate through files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith(".jpg"):
         # The last part "zzz" of the filename xxx_yyy_zzz is the direction control
        direction = filename.split('_')[-1] 
        
        # Construct the full path to the image
        image_path = os.path.join(directory_path, filename)
        
        # Append the data to the lists
        image_paths.append(image_path)
        directions.append(direction)

# Label encoding

In [3]:
label_encoder = LabelEncoder()

# Convert categorical values into integers
encoded_directions = label_encoder.fit_transform(directions)

# For neural networks, it is also recommended to apply one hot encoding
onehot_encoded_directions = to_categorical(encoded_directions)

ValueError: zero-size array to reduction operation maximum which has no identity

# Train/Test split
80/20 split is generally a good starting point.

In [3]:
X_train, X_valid, y_train, y_valid = train_test_split( image_paths, directions, test_size=0.2)
print(f"Training data: {len(X_train)}\nValidation data: {len(X_valid)}")

NameError: name 'image_paths' is not defined

# Image Augmentation
For artificially increasing the amount of data for training and testing. This can prevent overfitting.\
Geometric transformation – flipping, zooming, cropping, panning...\
Color space transformatinos  –change RGB channelge

In [ ]:
def zoom(image):
    # Specify the zoom factor
    zoom_factor = 1.5
    
    # Get the image dimensions
    height, width, _ = image.shape
    
    # Calculate the new dimensions after zooming
    new_height = int(height * zoom_factor)
    new_width = int(width * zoom_factor)
    
    # Resize the image and return
    return cv2.resize(image, (new_width, new_height))

def flip(image, direction):
    # Flip the image
    image = cv2.flip(image,1)

    # If car is making a turn flip the direction label as well
    if direction == "leftTurn":
        direction == "rightTurn"
    elif direction == "rightTurn":
        direction == "leftTurn"
    elif direction == "leftinPlaceTurn":
        direction == "rightInPlaceTurn"
    elif direction == "rightInPlaceTurn":
        direction == "leftInPlaceTurn"
   
    return image, steering_angle

def adjust_brightness(image, max_delta=30):
    # Randomly adjust the brightness of the image
    delta = np.random.uniform(-max_delta, max_delta)
    image = cv2.add(image, np.array([delta]))
    return image

def translate_shift(image, max_translation=20):
    # Randomly translate (shift) the image horizontally and/or vertically
    rows, cols, _ = image.shape
    dx = np.random.uniform(-max_translation, max_translation)
    dy = np.random.uniform(-max_translation, max_translation)
    translation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])
    image = cv2.warpAffine(image, translation_matrix, (cols, rows))
    return image

def perform_random_augmentation(image, direction):
    # Randomly pick and apply augmentation to the image
    augmentations = [
        zoom, flip, adjust_brightness, translate_shift
    ]

    selected_augmentations = np.random.choice(augmentations, np.random.randint(1, len(augmentations)), replace=False)

    for augmentation in selected_augmentations:
        if augmentation == flip:
            image, direction = augmentation(image, direction)
        else:
            image = augmentation(image)

    return image, direction

# Preprocess images

In [ ]:
def preprocess_images(image_paths, directions, target_size=(224, 224)):
    target_size = (224, 224)
    images = []
    augmented_directions = []
    for path, direction in zip(image_paths, directions)
        # Load image using cv2
        image = cv2.imread(path)
        
        # Resize the image to the target size
        image = cv2.resize(image, target_size)
        
        # Perform random augmentation
        image, augmented_direction = perform_random_augmentation(image, direction)

        # Normalize image
        image = cv2.normalize(img, None, 0, 1.0, cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        
        # Append the processed image and augmented classifier to the list
        images.append(image)
        augmented_directions.append(augmented_direction)
    
    return np.array(images), np.array(augmented_directions)

In [ ]:
# Apply preprocessing
X_train_processed, y_train_processed = preprocess_images(X_train, y_train)
X_valid_processed, y_valid_processed = preprocess_images(X_valid, y_valid)

# Model
## Creating the model

In [ ]:
num_classes = len(directions)

model = Sequential([
    layers.Conv2D(8, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling2D(),
    # Dropout layer to reduce overfitting
    layers.Dropout(0.2),
    # Fully connected layer
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

return model

## Compile model

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])

## Model summary

In [ ]:
model.summary()

# Train model

In [ ]:
history = model.fit(X_train_processed, y_train_processed, validation_data=(X_valid_processed, y_valid_processed), epochs=10)

# Plot accuracy over epochs

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss over epochs

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Classification metrics

In [ ]:
# Evaluate the model on the validation set
y_pred_valid = model.predict(X_valid_processed)
y_pred_valid_binary = (y_pred_valid > 0.5).astype(int)

# Calculate and print classification metrics
accuracy_valid = accuracy_score(y_valid_processed, y_pred_valid_binary)
precision_valid = precision_score(y_valid_processed, y_pred_valid_binary)
recall_valid = recall_score(y_valid_processed, y_pred_valid_binary)
f1_valid = f1_score(y_valid_processed, y_pred_valid_binary)

print(f"Validation Accuracy: {accuracy_valid}")
print(f"Validation Precision: {precision_valid}")
print(f"Validation Recall: {recall_valid}")
print(f"Validation F1 Score: {f1_valid}")